In [ ]:
!pip install fastai --upgrade
!pip install labelbox

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from fastai.vision.all import *

Helper functions defined inside the exported FastAI model.

These must be EXACTLY the same as the ones defined in the training notebook.

In [ ]:
def get_image_path(img):
  return img

def get_image_classes(img):
  classifs = img2classifs.get(img)
  if classifs is None:
    return []
  else:
    return classifs.split()

Load the model from the FastAI .pkl file

In [ ]:
learner = load_learner("/content/drive/MyDrive/480ChevronDrone/models/third-multiclass.pkl")

Connect to LabelBox

In [ ]:
import labelbox

#@title Raw fields
labelbox_api_key = ""#@param {type:"string"}
labelbox_client = labelbox.Client(api_key=labelbox_api_key)
labelbox_project = labelbox_client.get_projecimage_data = label.data_row()
labelbox_dataset = labelbox_client.get_dataset("ckgy008m67eku0784rld6g3lo")

Make directory to contain some test images

In [ ]:
from pathlib import Path

output_dir = '/content/test_data/'
Path(output_dir).mkdir(parents=True, exist_ok=True)

Get all image data from LabelBox

In [ ]:
output_dir = '/content/test_data/'

image_info = []

for label in labelbox_project.labels():
  image_data = label.data_row()
  file_name = image_data.external_id
  image_info.append((image_data.row_data, file_name))

I looked through some images and found these two that I thought would be an ok fit for this as an example

In [ ]:
test_images = []
test_images.append(image_info[0])
test_images.append(image_info[691])

Download all test images that were deemed good for this example

In [ ]:
for img in test_images:
  img_link = img[0]
  img_name = img[1]
  response = urllib.request.urlretrieve(img_link, output_dir + img_name)

Get dependencies for decoding

In [ ]:
!apt install libzbar0
!pip install pyzbar

import cv2
from pyzbar.pyzbar import decode, ZBarSymbol
from PIL import Image

Custom decode methods for OpenCV and PyZBar. More testing is needed on which of the two are superior.

In [ ]:
def opencv_decode(img_path):
  img = cv2.imread(img_path)
  retval, points = cv2.QRCodeDetector().detect(img)
  print('Was a QR code found?', retval)
  if retval:
      data, straight_code = cv2.QRCodeDetector().decode(img, points)
      if len(data) > 0:
        print('Detection data: \n', data)
      else:
        print('QR tag could not be decoded')

def pyzbar_decode(img_path):
  img = Image.open(img_path)
  print('Detection data: ', decode(img))

In [ ]:
for img in test_images:
  img_name = img[1]
  print(f'Detecting codes in {img_name}')
  res = learner.predict(output_dir + img_name)
  if 'qr_code' in res[0]:
    print('QR code found. Decoding...')

    print('\nAttempting to decode with OpenCV')
    print('--------------------------------')
    opencv_decode(output_dir + img_name)
    print('--------------------------------')

    print('\nAttempting to decode with PyZBar')
    print('--------------------------------')
    pyzbar_decode(output_dir + img_name)
    print('--------------------------------\n')

Detecting codes in DJI_0321-0h0m36s903.png


QR code found. Decoding...

Attempting to decode with OpenCV
--------------------------------
Was a QR code found? False
--------------------------------

Attempting to decode with PyZBar
--------------------------------
Detection data:  []
--------------------------------

Detecting codes in DJI_0324-0h0m7s941.png


QR code found. Decoding...

Attempting to decode with OpenCV
--------------------------------
Was a QR code found? True
QR tag could not be decoded
--------------------------------

Attempting to decode with PyZBar
--------------------------------
Detection data:  []
--------------------------------

